In [1]:
!pip install simpleai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.4/94.4 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for simpleai: filename=simpleai-0.8.3-py3-none-any.whl size=100984 sha256=f062a2a69febaa15086af97fa6fab87c4655e54c298e8eaebe2c14fd0eb6d792
  Stored in directory: /root/.cache/pip/wheels/91/0c/38/421d7910e7bc59b97fc54f490808bdb1097607d83d1a592865
Successfully built simpleai


In [2]:
class Food(object):
    def __init__(self, location):
        self.location = location

class Water(object):
    def __init__(self, location):
        self.location = location

class Agent:
    def __init__(self, name, location=0):
        self.location = location
        self.name = name

    def movedown(self):
        '''increase the location of the agent'''
        self.location += 1
        print("Agent {}: moves down to location {}.".format(self.name, self.location))

    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            print("Agent {}: Ate food at {}.".format(self.name, self.location))
            return True
        return False

    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            print("Agent {}: Drank water at {}.".format(self.name, self.location))
            return True
        return False

    def program(self, percepts):
        '''Returns an action based on it's percepts'''
        for p in percepts:
            if isinstance(p, Food):
                return 'eat'
            elif isinstance(p, Water):
                return 'drink'
        return 'move down'

In [3]:
from simpleai.environments import Environment

class Park(Environment):
    def __init__(self, agents, initial_state):
        super(Park, self).__init__(agents, initial_state)

    def do_action(self, state, action, agent):
        '''changes the state of the environment based on what the agent does.'''
        if action == "move down":
            agent.movedown()
        elif action == "eat":
            items = self.list_things_at(state, agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]): #Have the dog pick eat the first item
                    self.state.remove(items[0]) #Delete it from the Park after.
        elif action == "drink":
            items = self.list_things_at(state, agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): #Have the dog drink the first item
                    self.state.remove(items[0]) #Delete it from the Park after.
        return self.state

    def is_completed(self, state):
        '''By default, we're done when we can't find a live agent,
        but to prevent killing our cute dog, we will or it with when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water)
                             for thing in state)
        return no_edibles

    def percept(self, agent, state):
        '''prints & return a list of things that are in our agent's location'''
        things = self.list_things_at(state, agent.location)
        return things

    def list_things_at(self, state, location, tclass=object):
        things = [thing for thing in state
                  if thing.location == location and isinstance(thing, tclass)]
        return things

In [4]:
dog = Agent("Dog")
dogfood = Food(5)
water = Water(7)
park = Park([dog],[dogfood, water])

park.run(10)

Agent Dog: moves down to location 1.
Agent Dog: moves down to location 2.
Agent Dog: moves down to location 3.
Agent Dog: moves down to location 4.
Agent Dog: moves down to location 5.
Agent Dog: Ate food at 5.
Agent Dog: moves down to location 6.
Agent Dog: moves down to location 7.
Agent Dog: Drank water at 7.
